In [1]:
# imports
# system related
import sys
import subprocess

# basics
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep

# snap7 and logging
import snap7 as s7


print('imports succesful')

imports succesful


## Connecting to plc

In [2]:
# connecting to plc

plc = s7.client.Client()
plc.connect('192.168.0.19', 0, 1, )  # ip, rack, slot

print(f'connection status: {plc.get_connected()}')

connection status: True


## Defining get & set functions for plc variables

In [6]:
# Defining read/write functions

def get_bool(db_number, start_offset, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    value = s7.util.get_bool(data, 0, bit_offset)
    # print(f'db number: {db_number}, bit: {start_offset}, has boolean value: {value}')
    return value
    
def set_bool(db_number, start_offset, value, bit_offset=0):
    data = plc.db_read(db_number, start_offset, 1) # read 1 bite
    s7.util.set_bool(data, 0, bit_offset, value)
    plc.db_write(db_number, start_offset, data)
    return None


def get_real(db_number, start_offset):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    real = s7.util.get_real(data, 0)
    return real

def set_real(db_number, start_offset, value):
    data =  plc.db_read(db_number, start_offset, 4) # read 4 bites
    s7.util.set_real(data, 0, value)
    plc.db_write(db_number, start_offset, data)
    return None



    


## Creating RPMs to loop thru

In [58]:
# creating random values to loop thru

############ CHECK THESE VAls!!

min_rpm = 34
max_rpm = 55 ## ajust per nozzle! 

num_values = 150
rpm_interval = 5  # s
T_choices = [250]
print(T_choices)

rpms = [min_rpm, max_rpm] * int(num_values/2)
print(f'interval = {rpm_interval}s')
print(rpms)



[250]
interval = 5s
[34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55, 34, 55]


## Defining DB number and start offsets for the variables

In [59]:
db_py = 50

# defining start_offsets (so) for the variables in the DB50
so_rpm = 0
so_hz1 = 4
so_hz2 = 8
so_hz3 = 12
so_hz4 = 16


In [61]:
# for loop true random rpms
tot = len(rpms)*len(T_choices)
try:
    for j, T in enumerate(T_choices):
        set_real(db_py, so_hz1, T)
        set_real(db_py, so_hz2, T)
        set_real(db_py, so_hz3, T)
        set_real(db_py, so_hz4, T)
        for i, rpm in enumerate(rpms):
            prog=j*len(rpms)+i +1
            set_real(db_py, so_rpm, rpm)
            print(f'Progress: |{int(100*prog/tot)*"█" + int(100*(1-(prog/tot)))*"-"} | {np.round(prog/tot*100 ,1)}% | {prog}/{tot} | ETA: {np.round((tot-prog)*rpm_interval/60,1)} min | RPM: {rpm} | T: {j+1}/{len(T_choices)} | T_set = {T}  ' , end='\r')
            sleep(rpm_interval)
        
except KeyboardInterrupt:
    print('Sending RPMs interupted by keystroke                                                         It came this far: ')

print('all rpms have been sent, the job is finished')

all rpms have been sent, the job is finished███████████████████████████████████████████████████████████████████ | 100.0% | 150/150 | ETA: 0.0 min | RPM: 55 | T: 1/1 | T_set = 250  


In [98]:
set_real(db_py, so_hz1, 250)
set_real(db_py, so_hz2, 250)
set_real(db_py, so_hz3, 250)
set_real(db_py, so_hz4, 250)